In [32]:
# ProjectLib (if used)

# Using IBM Cloud Data Engine

<div class="pull-left"><left><img style="float: right;" src="http://developer.ibm.com/clouddataservices/wp-content/uploads/sites/85/2018/01/ibm-cloud-object-storage-logo-small.png" width="100" margin=50></left></div>
<div style="text-align:center">
Data Engine is the central data lake service of IBM Cloud. In addition to scalable serverless data processing on data in Cloud Object Storage (COS), it provides a table catalog service that is compatible with Hive Metastore. It allows to run ANSI SQL on Parquet, CSV, JSON, ORC and AVRO data sets. You can use it to run your analytic queries, and you can use it to conduct complex transformations and write the result in any desired data format, partitioning and layout. Data Engine is based on Apache Spark SQL as the query engine in the background. This means you do not have to provision any Apache Spark instance or service. For more background information, check out the Data Engine <a href="https://console.bluemix.net/docs/services/sql-query/getting-started.html#getting-started-tutorial" target="_blank">documentation</a>.
 
As mentioned in that documentation, you can make a direct SQL query using the Data Engine Web UI or using a simple Python client (like the IBM Watson Studio Notebook) is sufficient.
<br><br></div>

This notebook is meant to be a generic starter to use the Data Engine API in order to run SQL statements in a programmatic way. It uses the <a href="https://github.com/IBM-Cloud/sql-query-clients/tree/master/Python" target="_blank">ibmcloudsql</a> Python library for this purpose.
For information about using **ibmcloudsql** Python library, check out the <a href="https://ibm-cloud.github.io/sql-query-clients/intro.html#ibmcloudsql" target="_blank">documentation</a>.

The notebook also demonstrates how you can combine Data Engine with visualization libraries such as PixieDust. The notebook has been verified to work with Python 3.5. As mentioned above it does not require a Spark service bound to the notebook.

<a id="toc"></a>
## Table of contents
1. [Setup libraries](#setup)<br>
2. [Configure SQL Query](#configure)<br>
    2.1 [Using the project bucket](#projectbucket)<br>
    2.2 [Setting SQL Query parameters](#parameters)<br>
3. [Understanding your data](#schema)<br>
3. [Creating Your SQL](#sql)<br>
5. [Running Your SQL Statement](#run)<br>
    5.1 [submit_sql, submit](#lowlevel)<br>
    5.2 [run_sql, run](#run_sql)<br>
    5.3 [execute_sql](#execute_sql)<br>
5. [Work with result](#results) (via job_id)<br>
6. [Work with objects on COS](#cos) (via COS URL)<br>
6. [Running ETL SQLs](#etl)<br>
    8.1. [Fast run with execute_sql](#etl_execute_sql)<br>
    8.2. [Catalog table](#etl_catalog)<br>
6. [Paginated SQL Results](#pagination)<br>
7. [Automate SQL Statements](#many_queries)<br>
8. [List recent SQL submissions](#joblist)<br>
9. [Next steps](#next)<br>

### <a id="setup"></a> 1. Setup libraries

Run the following cell at least once in your notebook environment in order to install required packages, such as the SQL Query client library:

In [13]:
%time
!pip install pyarrow
!pip install sqlparse

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs
You should consider upgrading via the '/Users/tmhoangtus.ibm.com/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/tmhoangtus.ibm.com/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
#pip install ibmcloudsql
!pip install backoff 
!pip install numpy 
#import sys 
#print(sys.path)
import numpy 

In [1]:
import ibmcloudsql
import sqlparse 
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter

from pixiedust.display import *
import pandas as pd
targeturl=''

lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='vim')


Pixiedust database opened successfully


### <a id="configure"></a> 2. Configure SQL Query

1. You need an **API key** for an IBM cloud identity. This single key provides you accesses to both your Cloud Object Storage (COS) bucket for writing SQL results and to your Data Engine instance. To create API keys log on to the IBM Cloud console and go to <a href="https://console.bluemix.net/iam/#/apikeys" target="_blank">Manage->Access (IAM): then select API Keys</a>, click the `Create an IBM Cloud API key` button, give the key a custom name and click `Create`. In the next dialog click `Show` and copy the key to your clipboard and paste it below in this notebook.





2. You need the **instance CRN** for the Data Engine instance. If you don't have a Data Engine instance created yet, <a href="https://console.bluemix.net/catalog/services/sql-query" target="_blank">create one</a> first. If you already have one, you can find it in the <a href="https://console.bluemix.net/dashboard/apps" target="_blank">IBM Cloud console dashboard</a>. Make sure you select the right `Resource Groups` for **Group**. In the section `Services` you can see different types of services (created for the selected Group), and SQL service instances have the icon like the one at the top of this notebook. Select the instance of Data Engine that you want to use. In the Data Engine dashboard page that opens up you find a section titled **Overview** with *Deployment Details* and copy the text after **CRN**. Click the button to copy the CRN into your clipboard and paste it here into the notebook. 

3. You need to specify the location on COS where your **query results** should be written. This is because Data Engine instance, to process an ETL SQL statement, needs to store queried data on COS.
This COS location comprises three parts of information that you can find in the Cloud Object Storage UI for your instance in the IBM Cloud console. You need to provide it as a **URL** using the format `cos://<endpoint>/<bucket>/[<prefix>]`. You have the option to use the cloud object storage **bucket that is associated with your Watson Studio project**. In this case, execute the following section before you proceed.  
<br/>

#### <a id="projectbucket"></a> 2.1 Using the project bucket
**Only** follow the instructions in this section when you want to write your SQL query results to the bucket that has been created for the project for which you have created this notebook. In any other case proceed directly with section **2.2**.
<br><br>


__Inserting the project token__:  
Click the `More` option in the toolbar above (the three stacked dots) and select `Insert project token`.
 * If you haven't created an access token for this project before, you will see a dialog that asks you to create one first. Follow the link to open your project settings, scroll down to `Access tokens` and click `New token`. Give the token a custom name and make sure you select `Editor` as `Access role for project`. After you created your access token you can come back to this notebook, select the empty cell below and again select `Insert project token` from the toolbar at the top.
[//]: # 
This will add a new cell at the top of your notebook with content that looks like this:
```
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='<some id>', project_access_token='<some access token>')
pc = project.project_context
```
Leave that cell content as inserted and run the cell. Then then proceed with the following cell below:

In [2]:
cos_bucket = project.get_metadata()['entity']['storage']['properties']
targeturl="cos://" + cos_bucket['bucket_region'] + "/" + cos_bucket['bucket_name'] + "/"

NameError: name 'project' is not defined

#### <a id="parameters"></a> 2.2 Setting the SQL Query parameters

There are three optional arguments: 

1. max_concurrent_jobs (from version 0.4)
2. max_tries
3. target_cos_url (from version 0.4)

that you may want to <a href="https://ibm-cloud.github.io/sql-query-clients/ibmcloudsql.html#ibmcloudsql.SQLQuery.SQLQuery">use</a>, if needed. It is recommended to always provide the value for the third one (the COS storage for queried data), as some new APIs use it implicitly. 

In [2]:
# TODO: Fix this
apikey = None 
instnacecrn = None

if targeturl == '':
    targeturl=input('Enter target URL for SQL results: ')
else:
    targeturl=input('Enter target URL for SQL results (leave empty to use ' + targeturl + '): ') or targeturl

if apikey is None or instnacecrn is None:
    try: 
        import test_credentials 
        apikey = test_credentials.apikey
        instnacecrn = test_credentials.instnacecrn
        targeturl = test_credentials.targeturl
    except Exception:
        pass

sqlClient = ibmcloudsql.SQLQuery(apikey, instnacecrn, client_info='Data Engine Starter Notebook',  target_cos_url=targeturl)
#sqlClient.configure()  # use this if you want to change the API key or Data Engine CRN later

sqlClient.logon()
print('\nYour Data Engine web console link:\n')
sqlClient.sql_ui_link()


Your SQL Query web console link:

https://sql-query.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/57dd42208dce4fd88998d8976d0edfa5:988ac7b3-11bf-4eb6-b37c-224b738bfb61::


'https://sql-query.cloud.ibm.com/sqlquery/?instance_crn=crn:v1:bluemix:public:sql-query:us-south:a/57dd42208dce4fd88998d8976d0edfa5:988ac7b3-11bf-4eb6-b37c-224b738bfb61::'

### <a id="schema"></a> 3. Understanding your data
[Home](#toc)

Since `ibmcloudsql` version 0.4, you can directly query the <a href="https://ibm-cloud.github.io/sql-query-clients/ibmcloudsql.html#ibmcloudsql.SQLQuery.SQLQuery.get_schema_data">schema of the data</a>. 

In [3]:
sqlClient.get_schema_data("cos://us-geo/sql/employees.parquet", type="parquet", dry_run=True)

sqlClient.get_schema_data("cos://us-geo/sql/employees.parquet", type="parquet")



        SELECT * FROM DESCRIBE(cos://us-geo/sql/employees.parquet STORED AS PARQUET)
        INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS JSON
        


,name,nullable,type
0,employeeID,True,integer
1,lastName,True,string
2,firstName,True,string
3,title,True,string
4,titleOfCourtesy,True,string
5,birthDate,True,timestamp
6,hireDate,True,timestamp
7,address,True,string
8,city,True,string
9,region,True,string


### <a id="sql"></a> 4. Your SQL Statement
[Home](#toc)

You provide the SQL statement in the form of a string. To author your own SQL query, use the interactive Data Engine web console (**link above**) of your Data Engine service instance.

In [4]:

sql=input('Enter your SQL statement (leave empty to use a simple sample SQL)')
if sql == '':
    sql='SELECT o.OrderID, c.CompanyName, e.FirstName, e.LastName FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
         cos://us-geo/sql/employees.parquet STORED AS PARQUET e, cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
         WHERE e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01" \
         ORDER BY c.CompanyName'
if " INTO " not in sql:
    sql += ' INTO {} STORED AS CSV'.format(targeturl)
    
formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_sql, lexer, formatter)
from IPython.core.display import display, HTML
print('\nYour SQL statement is:\n')
print(result)


Your SQL statement is:

SELECT o.OrderID,
	c.CompanyName,
	e.FirstName,
	e.LastName
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/employees.parquet STORED AS PARQUET e,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE e.EmployeeID = o.EmployeeID
		AND c.CustomerID = o.CustomerID
		AND o.ShippedDate > o.RequiredDate
		AND o.OrderDate > "1998-01-01"
ORDER BY c.CompanyName INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS CSV



Since version 0.4, *ibmcloudsql* provides a new way to generate your SQL statement, using the functionality provided by the <a href="https://ibm-cloud.github.io/sql-query-clients/sql_magic.html">*sql_magic* module</a>, and these APIs are also accessible via the SQLQuery module. The documentation is provided via <a href="https://ibm-cloud.github.io/sql-query-clients/sql_magic.html#sqlmagic">this link</a>.

In the provided example above, we can generate it using the following code. The default `format_type` is **parquet**.

In [5]:
sqlClient.reset_()
(sqlClient.select_("o.OrderID, c.CompanyName, e.FirstName, e.LastName")
        .from_cos_("cos://us-geo/sql/orders.parquet", format_type="parquet", alias="o")
        .from_cos_("cos://us-geo/sql/employees.parquet", format_type="parquet", alias="e")
        .from_cos_("cos://us-geo/sql/customers.parquet", format_type="parquet", alias="c")
        .where_('e.EmployeeID = o.EmployeeID AND c.CustomerID = o.CustomerID AND o.ShippedDate > o.RequiredDate AND o.OrderDate > "1998-01-01"')
        .order_by_("c.CompanyName")
        .store_at_(targeturl, format_type="csv")
)
# sqlClient.store_at_(targeturl, format_type="csv")

sqlClient.print_sql()

if 0:
    formatted_sql = sqlClient.get_sql()
    lexer = get_lexer_by_name("sql", stripall=True)
    formatter = Terminal256Formatter(style='vim')
    result = highlight(formatted_sql, lexer, formatter)
    from IPython.core.display import display, HTML
    print(result)


SELECT o.OrderID,
       c.CompanyName,
       e.FirstName,
       e.LastName
FROM cos://us-geo/sql/orders.parquet STORED AS parquet o, cos://us-geo/sql/employees.parquet STORED AS parquet e, cos://us-geo/sql/customers.parquet STORED AS parquet c
WHERE e.EmployeeID = o.EmployeeID
  AND c.CustomerID = o.CustomerID
  AND o.ShippedDate > o.RequiredDate
  AND o.OrderDate > "1998-01-01"
ORDER BY c.CompanyName INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS CSV


### <a id="run"></a> 5. Running Your SQL Statement
[Home](#toc)


The following cell submits the above SQL statement and waits for it to finish before printing a sample of the result set. Remember that each Data Engine instance can process multiple queries but no more than a maximum capacity (default: 5). By default, the launching of the job fails if the instance is processing the maximum capacity. When you automate the launching of multiple jobs, there is a chance that the queue can be full, and to overcome that you can adjust `max_tries` option when creating `sqlClient` object. 

Since version 0.4, you have many options to launch a SQL query, as given in the <a href="https://ibm-cloud.github.io/sql-query-clients/sql_query.html#b-submit-sql-jobs">documentation</a>.

1. launch, and return the control immediately (along with job_id): `submit_sql`, [*new from version 0.4*] `submit()`
2. launch, wait (until the query is completed), and returns the queried data: `run_sql`
3. launch, wait (until the query is completed), and return a named tuple (data, job_id) [*new from version 0.4*: .execute_sql(), .run()] with `get_result` option controls if data is returned or not. Turning this on is not recommended if the queried data are very large, or you should use the `pagesize` option on [paginated result](#pagination).

NOTE: To run the SQL statement generated using `sql_magic`, we need to use .run() or .submit() method. 

Also, many APIs have the optional **`dry_run`** parameter for printing out the SQL statement, and no real job is launched/requested.



#### <a id="lowlevel"></a> 5.1 submit_sql(), submit()
This is the asynchronous submission mechanism and the status check method.
`submit_sql` runs the given SQL string; while `submit` runs the internally generated SQL string via `sql_magic` module.


In [6]:
%time
sqlClient.logon()
jobId = sqlClient.submit_sql(sql)
print("SQL query submitted and running in the background. jobId = " + jobId)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
SQL query submitted and running in the background. jobId = 8fd9e686-61af-4696-80a0-8e4e9922ea19


In [7]:
print("Job status for " + jobId + ": " + sqlClient.get_job(jobId)['status'])

Job status for 8fd9e686-61af-4696-80a0-8e4e9922ea19: running


Use the `wait_for_job()` method as a blocking call until your job has finished:

In [8]:
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
if job_status == 'failed':
    details = sqlClient.get_job(jobId)
    print("Error: {}\nError Message: {}".format(details['error'], details['error_message']))

Job 8fd9e686-61af-4696-80a0-8e4e9922ea19 terminated with status: completed


Use the `get_result()` method to retrieve a dataframe for the SQL result set:

In [11]:
result_df = sqlClient.get_result(jobId)
print("OK, we have a dataframe for the SQL result that has been stored by Data Engine in " + sqlClient.get_job(jobId)['resultset_location'])

OK, we have a dataframe for the SQL result that has been stored by SQL Query in cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/jobid=d6a19741-55a8-4c36-8fa3-97e02a00aa24


#### <a id="run_sql"></a> 5.2 run_sql(), run() 


`run_sql` runs the given SQL string; while `run` runs the internally generated SQL string via `sql_magic` module.


In [9]:
%time
result_df = sqlClient.run_sql(sql)
if isinstance(result_df, str):
    print(result_df)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


In [10]:
display(result_df)

,OrderID,CompanyName,FirstName,LastName
0,10924,Berglunds snabbköp,Janet,Leverling
1,11058,Blauer See Delikatessen,Anne,Dodsworth
2,10827,Bon app',Nancy,Davolio
3,11076,Bon app',Margaret,Peacock
4,11045,Bottom-Dollar Markets,Michael,Suyama
5,10970,Bólido Comidas preparadas,Anne,Dodsworth
6,11054,Cactus Comidas para llevar,Laura,Callahan
7,11008,Ernst Handel,Robert,King
8,11072,Ernst Handel,Margaret,Peacock
9,10816,Great Lakes Food Market,Margaret,Peacock


In [13]:
%time 
result = sqlClient.run(get_result=True) 
display(result)
# NOTE: result is a named tuple (.data, .job_id)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


RunSql(data=    OrderID                 CompanyName FirstName   LastName
0     10924          Berglunds snabbköp     Janet  Leverling
1     11058     Blauer See Delikatessen      Anne  Dodsworth
2     10827                    Bon app'     Nancy    Davolio
3     11076                    Bon app'  Margaret    Peacock
4     11045       Bottom-Dollar Markets   Michael     Suyama
5     10970   Bólido Comidas preparadas      Anne  Dodsworth
6     11054  Cactus Comidas para llevar     Laura   Callahan
7     11008                Ernst Handel    Robert       King
8     11072                Ernst Handel  Margaret    Peacock
9     10816     Great Lakes Food Market  Margaret    Peacock
10    11040     Great Lakes Food Market  Margaret    Peacock
11    11061     Great Lakes Food Market  Margaret    Peacock
12    10960            HILARION-Abastos     Janet  Leverling
13    11065           LILA-Supermercado     Laura   Callahan
14    11071           LILA-Supermercado     Nancy    Davolio
15    11039 

#### <a id="run_sql"></a> 5.3 execute_sql()

This is the new API that combines the behavior of `run_sql` and `submit_sql`, i.e. it is a synchronous version that returns job_id, and (optionally) data. 


In [11]:
%time
result = sqlClient.execute_sql(sql)
print(result)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
RunSql(data=None, job_id='fd7fb935-707b-4a9b-8a9d-bde0d188a4a1')


### <a id="results"></a> 6. Work with results

The queried data, apart from being returned to the client, is also stored permanently on COS in the form of objects.

This 
<a href="https://ibm-cloud.github.io/sql-query-clients/sql_query.html#c-work-with-query-results">link</a> provides different APIs that you can use to interact with queried result, via the `job_id`.  Here, we introduce some of them.

This section shows some useful methods to work with result objects of SQLs. The following cell lists the result objects of a given job_id.


In [15]:
result_df = sqlClient.list_results(jobId)
display(result_df)

,Object,LastModified,Size,StorageClass,Bucket,ObjectURL
0,jobid=d6a19741-55a8-4c36-8fa3-97e02a00aa24,2020-07-27 13:33:17.292000+00:00,0,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...
1,jobid=d6a19741-55a8-4c36-8fa3-97e02a00aa24/_SU...,2020-07-27 13:33:21.257000+00:00,0,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...
2,jobid=d6a19741-55a8-4c36-8fa3-97e02a00aa24/par...,2020-07-27 13:33:20.752000+00:00,1200,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...


You can delete the result set from Cloud Object Storage using the `delete_result()` method:

In [16]:
sqlClient.delete_result(jobId)

sqlClient.delete_result(result.job_id)


,Deleted Object
0,jobid=24b5489d-3e28-46d0-9f2d-84d5fe51766d
1,jobid=24b5489d-3e28-46d0-9f2d-84d5fe51766d/_SU...
2,jobid=24b5489d-3e28-46d0-9f2d-84d5fe51766d/par...


As can be seen the SQL job always produces a logical hierarchy of object names. The path specified in the URI of the `INTO` clause above is used as a logical root "folder", which is nothing else than an empty object. It will normally include a suffix containing the unique jobId that produced this result. But because the query above used the optional clause `JOBPREFIX NONE` this suffix is ommitted, which means each time the same target path will be used with that clause the previous list of objects in that logical folder will be overwritten.

In [17]:
sql="SELECT * FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET LIMIT 100 INTO {}first100orders.parquet JOBPREFIX NONE STORED AS PARQUET".format(targeturl)
jobId = sqlClient.submit_sql(sql)
sqlClient.wait_for_job(jobId)
sqlClient.list_results(jobId).head(100)

,Object,LastModified,Size,StorageClass,Bucket,ObjectURL
0,first100orders.parquet,2020-07-27 13:35:12.832000+00:00,0,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...
1,first100orders.parquet/_SUCCESS,2020-07-27 13:35:17.927000+00:00,0,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...
2,first100orders.parquet/part-00000-dce177b9-62c...,2020-07-27 13:35:16.932000+00:00,10271,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...


#### Exact target result
If the result of the SQL job was not written in a partitioned way, you can now optionally restructure the objects to a single object with the exact name that was specified in the `INTO` clause of the SQL job. Thisis achieved with the method `rename_exact_result()`:

In [18]:
sqlClient.rename_exact_result(jobId)
sqlClient.list_results(jobId).head(100)

,Object,LastModified,Size,StorageClass,Bucket,ObjectURL
0,first100orders.parquet,2020-07-27 13:35:27.684000+00:00,10271,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.clo...


As can be seen the result of the previous SQL is now exactly one object with the exact name from the path of the `INTO` clause.

In [19]:
jobId = sqlClient.submit_sql('SELECT * from cos://us-geo/sql/products.parquet STORED AS PARQUET LIMIT 5')
sqlClient.wait_for_job(jobId)
sqlClient.get_result(jobId).head()

,productID,productName,supplierID,categoryID,quantityPerUnit,unitPrice,unitsInStock,unitsOnOrder,reorderLevel,discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


### <a id="cos"></a> 7. Work with data on COS
[Home](#toc)

This 
<a href="https://ibm-cloud.github.io/sql-query-clients/cos.html#cosclient">link</a> provides different APIs that you can use to interact with data on COS. Of course, the queried result is also stored on COS; but here we interact via the COS URL. 

By default, the data is stored as part of the provided target COS URL, appended with `/jobid=<JOB_ID>`.  This is shown in the entry `resultset_location`. 
    

In [20]:
import pprint
job_details = sqlClient.get_job(jobId)
pprint.pprint(job_details)

{'bytes_read': 5843,
 'end_time': '2020-07-27T13:35:58.127Z',
 'job_id': 'd0d7205d-41bf-46d6-bf9e-8d92568891b4',
 'plan_id': 'ead0f7f5-0c96-40c0-9aae-63c4846d8188',
 'resultset_format': 'csv',
 'resultset_location': 'cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/jobid=d0d7205d-41bf-46d6-bf9e-8d92568891b4',
 'rows_read': 77,
 'rows_returned': 5,
 'statement': 'SELECT * from cos://us-geo/sql/products.parquet STORED AS '
              'PARQUET LIMIT 5',
 'status': 'completed',
 'submit_time': '2020-07-27T13:35:29.659Z',
 'user_id': 'tmhoangt@us.ibm.com'}


The COS URL structure can be divided into multiple parts, which can be extracted using <a href="https://ibm-cloud.github.io/sql-query-clients/ibmcloudsql.html#ibmcloudsql.cos.ParsedUrl">these APIs</a>.


In [21]:
print(job_details['resultset_location'])
print("===========")
x = sqlClient.analyze_cos_url(job_details['resultset_location'])
print(x)
print("===========")
print(x.endpoint)
print(x.bucket)
print(x.prefix)


cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/jobid=d0d7205d-41bf-46d6-bf9e-8d92568891b4
COSURL(endpoint='s3.us.cloud-object-storage.appdomain.cloud', bucket='timeseriessql-donotdelete-pr-flr5gmyhe2adeo', prefix='jobid=d0d7205d-41bf-46d6-bf9e-8d92568891b4')
s3.us.cloud-object-storage.appdomain.cloud
timeseriessql-donotdelete-pr-flr5gmyhe2adeo
jobid=d0d7205d-41bf-46d6-bf9e-8d92568891b4


### <a id="etl"></a> 8. Running ETL SQLs
The following ETL SQL statement joins two data sets from COS and writes the result to COS using **hive style partitioning** with two columns. This is done using `PARTITIONED BY` clause. 

In [12]:
etl_sql='SELECT OrderID, c.CustomerID CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax \
         EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, \
         ShipCity, ShipRegion, ShipPostalCode, ShipCountry FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
         cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
         WHERE c.CustomerID = o.CustomerID \
         INTO {}customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry, ShipCity)'.format(targeturl)
formatted_etl_sql = sqlparse.format(etl_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_etl_sql, lexer, formatter)
print('\nExample ETL Statement is:\n')
print(result)

print("==========================")
print("   from ibmcloudsql 0.4")
# a new way to generate the same SQL statement
sqlClient.reset_()
(sqlClient.select_("OrderID, c.CustomerID CustomerID, CompanyName, ContactName, ContactTitle, Address, City, Region, PostalCode, Country, Phone, Fax \
     EmployeeID, OrderDate, RequiredDate, ShippedDate, ShipVia, Freight, ShipName, ShipAddress, \
     ShipCity, ShipRegion, ShipPostalCode, ShipCountry")
        .from_cos_("cos://us-geo/sql/orders.parquet", alias="o")
        .from_cos_("cos://us-geo/sql/customers.parquet", alias="c")
        .where_('c.CustomerID = o.CustomerID')
        .order_by_("c.CompanyName")
        .store_at_(targeturl+"customer_orders", format_type="parquet")
        .partition_by_("(ShipCountry, ShipCity)")
)

sqlClient.print_sql()


Example ETL Statement is:

SELECT OrderID,
	c.CustomerID CustomerID,
	CompanyName,
	ContactName,
	ContactTitle,
	Address,
	City,
	Region,
	PostalCode,
	Country,
	Phone,
	Fax EmployeeID,
	OrderDate,
	RequiredDate,
	ShippedDate,
	ShipVia,
	Freight,
	ShipName,
	ShipAddress,
	ShipCity,
	ShipRegion,
	ShipPostalCode,
	ShipCountry
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE c.CustomerID = o.CustomerID INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders STORED AS PARQUET PARTITIONED BY (ShipCountry,
																																																																																																																																																			ShipCity)

   from ibmcloudsql 0.4
SELECT OrderID,
       c.CustomerID CustomerID,
       CompanyName,
       ContactName,
       ContactTitle,
       Address,
       City,
       Region,
       PostalCode,
       Country,
       Phone,
       Fax Emplo

In [13]:
import time

tic = time.perf_counter()
jobId = sqlClient.submit_sql(etl_sql)
print("SQL query submitted and running in the background. jobId = " + jobId)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))
else:
    result_df = sqlClient.get_result(jobId)
toc = time.perf_counter()
print(f"Total time  in {toc - tic:0.4f} seconds")

# since ibmcloudsql 0.4, you can use execute_sql()
tic = time.perf_counter()
result = sqlClient.execute_sql(etl_sql, get_result=True)
job_details = sqlClient.get_job(result.job_id)
job_status = job_details['status']
print("Job " + result.job_id + " terminated with status: " + job_status)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))
else:
    result_df = result.data
toc = time.perf_counter()
print(f"Total time  in {toc - tic:0.4f} seconds")


SQL query submitted and running in the background. jobId = 3dbb7914-a9f6-448d-b093-5f3e99a6e00f
Job 3dbb7914-a9f6-448d-b093-5f3e99a6e00f terminated with status: completed
Total time  in 71.1853 seconds
Job 11877eab-2e1b-4cda-865c-ff23210d79c2 terminated with status: completed
Total time  in 65.9625 seconds


Depending upon your SQL query, the queried result is stored on COS into one or many objects. 

The following cell uses the `get_cos_summary()` method to print a summary of the objects that have been written by the previous ETL SQL statement. Note the **total_volume** value. We will reference it for comparison in the next steps.

In [14]:
resultset_location = job_details['resultset_location']
sqlClient.get_cos_summary(resultset_location)

{'url': 'cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2/',
 'total_objects': 71,
 'total_volume': '401.7 KB',
 'oldest_object_timestamp': 'July 27, 2020, 15H:05M:24S',
 'newest_object_timestamp': 'July 27, 2020, 15H:05M:37S',
 'smallest_object_size': '0.0 B',
 'smallest_object': 'customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2/_SUCCESS',
 'largest_object_size': '7.1 KB',
 'largest_object': 'customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2/ShipCountry=UK/ShipCity=London/part-00015-78f8e4fe-1d83-4f81-a76f-f67472f314bb-attempt_20200727150525_0006_m_000015_106.c000.snappy.parquet'}

The following cell uses the `list_results()` method to print a list of the objects that have been written by the above ETL SQL statement. Note the partition columns and their values being part of the object names now. This naming convention is known as **hive style partitioning**. This type of partitioning is the basis for optimizing SQL queries using predicates that match with the partitioning columns.

In [15]:
pd.set_option('display.max_colwidth', -1)
result_objects_df = sqlClient.list_results(jobId)
print("List of objects written by ETL SQL:")
result_objects_df.head(200)

List of objects written by ETL SQL:


,Object,LastModified,Size,StorageClass,Bucket,ObjectURL
0,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f,2020-07-27 15:04:12.746000+00:00,0,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f
1,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Argentina/ShipCity=Buenos Aires/part-00065-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150423_0006_m_000065_166.c000.snappy.parquet,2020-07-27 15:04:23.663000+00:00,6412,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Argentina/ShipCity=Buenos Aires/part-00065-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150423_0006_m_000065_166.c000.snappy.parquet
2,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Austria/ShipCity=Graz/part-00096-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150425_0006_m_000096_175.c000.snappy.parquet,2020-07-27 15:04:25.747000+00:00,6094,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Austria/ShipCity=Graz/part-00096-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150425_0006_m_000096_175.c000.snappy.parquet
3,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Austria/ShipCity=Salzburg/part-00015-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150418_0006_m_000015_63.c000.snappy.parquet,2020-07-27 15:04:18.908000+00:00,5636,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Austria/ShipCity=Salzburg/part-00015-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150418_0006_m_000015_63.c000.snappy.parquet
4,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Belgium/ShipCity=Bruxelles/part-00180-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150430_0006_m_000180_198.c000.snappy.parquet,2020-07-27 15:04:30.239000+00:00,5660,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Belgium/ShipCity=Bruxelles/part-00180-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150430_0006_m_000180_198.c000.snappy.parquet
...,...,...,...,...,...,...
67,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Venezuela/ShipCity=Barquisimeto/part-00095-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150425_0006_m_000095_174.c000.snappy.parquet,2020-07-27 15:04:25.595000+00:00,6202,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Venezuela/ShipCity=Barquisimeto/part-00095-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150425_0006_m_000095_174.c000.snappy.parquet
68,customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Venezuela/ShipCity=Caracas/part-00084-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150424_0006_m_000084_172.c000.snappy.parquet,2020-07-27 15:04:25.062000+00:00,5526,STANDARD,timeseriessql-donotdelete-pr-flr5gmyhe2adeo,cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=3dbb7914-a9f6-448d-b093-5f3e99a6e00f/ShipCountry=Venezuela/ShipCity=Caracas/part-00084-126f7412-647d-4cf5-9069-f926624d596a-attempt_20200727150424_0006_m_000084_172.c000.snappy.parquet
69,cust

Now let's take a look at the result data with the `get_result()` method. Note that the result dataframe contains the two partitioning columns. The values for these have been put together by get_result() from the object names above because in hive style partitioning the partition column values are not stored in the objects but rather in the object names.

In [16]:
sqlClient.get_result(jobId).head(100)

,OrderID,CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,...,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipRegion,ShipPostalCode,ShipCountry,ShipCity
0,10409,OCEAN,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,NULL,1010,Argentina,...,1997-02-06 06:00:00,1997-01-14 00:00:00.000,1,29.83,Océano Atlántico Ltda.,Ing. Gustavo Moncada 8585 Piso 20-A,NULL,1010,Argentina,Buenos Aires
1,10448,RANCH,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,NULL,1010,Argentina,...,1997-03-17 06:00:00,1997-02-24 00:00:00.000,2,38.82,Rancho grande,Av. del Libertador 900,NULL,1010,Argentina,Buenos Aires
2,10521,CACTU,Cactus Comidas para llevar,Patricio Simpson,Sales Agent,Cerrito 333,Buenos Aires,NULL,1010,Argentina,...,1997-05-27 05:00:00,1997-05-02 00:00:00.000,2,17.22,Cactus Comidas para llevar,Cerrito 333,NULL,1010,Argentina,Buenos Aires
3,10531,OCEAN,Océano Atlántico Ltda.,Yvonne Moncada,Sales Agent,Ing. Gustavo Moncada 8585 Piso 20-A,Buenos Aires,NULL,1010,Argentina,...,1997-06-05 05:00:00,1997-05-19 00:00:00.000,1,8.12,Océano Atlántico Ltda.,Ing. Gustavo Moncada 8585 Piso 20-A,NULL,1010,Argentina,Buenos Aires
4,10716,RANCH,Rancho grande,Sergio Gutiérrez,Sales Representative,Av. del Libertador 900,Buenos Aires,NULL,1010,Argentina,...,1997-11-21 06:00:00,1997-10-27 00:00:00.000,2,22.57,Rancho grande,Av. del Libertador 900,NULL,1010,Argentina,Buenos Aires
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,10261,QUEDE,Que Delícia,Bernardo Batista,Accounting Manager,Rua da Panificadora,12Rio de Janeiro,RJ,02389-673,Brazil,...,1996-08-16 05:00:00,1996-07-30 00:00:00.000,2,3.05,Que Delícia,Rua da Panificadora 12,RJ,02389-673,Brazil,Rio de Janeiro
3,10287,RICAR,Ricardo Adocicados,Janete Limeira,Assistant Sales Agent,Av. Copacabana 267,Rio de Janeiro,RJ,02389-890,Brazil,...,1996-09-19 05:00:00,1996-08-28 00:00:00.000,3,12.76,Ricardo Adocicados,Av. Copacabana 267,RJ,02389-890,Brazil,Rio de Janeiro
4,10291,QUEDE,Que Delícia,Bernardo Batista,Accounting Manager,Rua da Panificadora,12Rio de Janeiro,RJ,02389-673,Brazil,...,1996-09-24 05:00:00,1996-09-04 00:00:00.000,2,6.40,Que Delícia,Rua da Panificadora 12,RJ,02389-673,Brazil,Rio de Janeiro
5,10299,RICAR,Ricardo Adocicados,Janete Limeira,Assistant Sales Agent,Av. Copacabana 267,Rio de Janeiro,RJ,02389-890,Brazil,...,1996-10-04 05:00:00,1996-09-13 00:00:00.000,2,29.76,Ricardo Adocicados,Av. Copacabana 267,RJ,02389-890,Brazil,Rio de Janeiro


The following cell runs a new **optimized SQL** query against the **partitioned data** that has been produced by the previous ETL SQL statement. The query uses `WHERE` predicates on the columns that have been used to partition the results in the ETL job. The query will physically only read the objects that match these predicate values.

In [17]:
optimized_sql='SELECT * FROM {} STORED AS PARQUET WHERE ShipCountry = "Austria" AND ShipCity="Graz" \
               INTO {} STORED AS PARQUET'.format(resultset_location, targeturl)
formatted_optimized_sql = sqlparse.format(optimized_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_optimized_sql, lexer, formatter)
print('\nRunning SQL against the previously produced hive style partitioned objects as input:\n')
print(result)

jobId = sqlClient.submit_sql(optimized_sql)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))


Running SQL against the previously produced hive style partitioned objects as input:

SELECT *
FROM cos://s3.us.cloud-OBJECT-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2 STORED AS PARQUET
WHERE ShipCountry = "Austria"
		AND ShipCity="Graz" INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS PARQUET

Job daa67280-6d4e-4c08-88e9-25b443d7d1d3 terminated with status: completed


The following cell uses the `get_job()` method in order to show the job details of the just run optimized SQL that leverages hive style partitioning. Note the **bytes_read** value that is significantly lower than the **total_volume** value of the data in the queries data set. This does increase query performance and lower the query cost.

In [18]:
sqlClient.get_job(jobId)

{'job_id': 'daa67280-6d4e-4c08-88e9-25b443d7d1d3',
 'status': 'completed',
 'statement': 'SELECT * FROM cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2 STORED AS PARQUET WHERE ShipCountry = "Austria" AND ShipCity="Graz"                INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS PARQUET',
 'plan_id': 'ead0f7f5-0c96-40c0-9aae-63c4846d8188',
 'submit_time': '2020-07-27T15:06:18.650Z',
 'resultset_location': 'cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/jobid=daa67280-6d4e-4c08-88e9-25b443d7d1d3',
 'rows_returned': 30,
 'rows_read': 30,
 'bytes_read': 6090,
 'resultset_format': 'parquet',
 'end_time': '2020-07-27T15:06:32.728Z',
 'user_id': 'tmhoangt@us.ibm.com'}

#### <a id="etl_catalog"></a> 8.2 catalog table

Data Engine provides the option to access to data on COS using the traditional table name.

Since ibmcloudsql version 0.4, we can use the APIs listed in <a href="https://ibm-cloud.github.io/sql-query-clients/sql_query.html#f-manage-table-catalog"> the link</a> to create such table name. 

NOTE: If the data is created using **hive style partitioning** (i.e. PARTITIONED BY clause), we need to create the table using `create_partitioned_table()`
Otherwise, we use `create_table()` API. 


In [19]:
table_name="test"
print(resultset_location)
sqlClient.create_partitioned_table(table_name, cos_url=resultset_location, format_type="parquet", force_recreate=True)

print(sqlClient.show_tables())

result = sqlClient.describe_table(table_name)
print(result.data)
#print(result.job_id)


cos://s3.us.cloud-object-storage.appdomain.cloud/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/customer_orders/jobid=11877eab-2e1b-4cda-865c-ff23210d79c2
  tableName
0  test    


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
print(result.data)

col_name  data_type  comment
0   OrderID                  int        NaN    
1   CustomerID               string     NaN    
2   CompanyName              string     NaN    
3   ContactName              string     NaN    
4   ContactTitle             string     NaN    
5   Address                  string     NaN    
6   City                     string     NaN    
7   Region                   string     NaN    
8   PostalCode               string     NaN    
9   Country                  string     NaN    
10  Phone                    string     NaN    
11  EmployeeID               string     NaN    
12  OrderDate                timestamp  NaN    
13  RequiredDate             timestamp  NaN    
14  ShippedDate              string     NaN    
15  ShipVia                  int        NaN    
16  Freight                  double     NaN    
17  ShipName                 string     NaN    
18  ShipAddress              string     NaN    
19  ShipRegion               string     NaN    
20  ShipPos

In [27]:
sqlClient.reset_()
(sqlClient.select_("*")
        .from_table_(table_name)
        .where_('ShipCountry = "Austria" AND ShipCity="Graz"')
        .store_at_(targeturl, format_type="parquet")
)

sqlClient.print_sql()

result=sqlClient.run(get_result=True) 

print("====================")
print(result.job_id)
print(result.data)


if 0:
    jobId = sqlClient.submit()
    job_status = sqlClient.wait_for_job(jobId)
    print("Job " + jobId + " terminated with status: " + job_status)
    job_details = sqlClient.get_job(jobId)
    if job_status == 'failed':
        print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))

SELECT *
FROM test
WHERE ShipCountry = "Austria"
  AND ShipCity="Graz" INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/ STORED AS PARQUET
bc3bd1c6-3d2e-4c9b-983d-54ebd95492af
    OrderID CustomerID   CompanyName    ContactName   ContactTitle  \
0   10258    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
1   10263    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
2   10351    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
3   10368    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
4   10382    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
5   10390    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
6   10402    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
7   10403    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
8   10430    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
9   10442    ERNSH      Ernst Handel  Roland Mendel  Sales Manager   
10  10514    ERNSH      Ernst Handel  Rol

### <a id="pagination"></a> 9. Paginated SQL Results
[Home](#toc)

Paginated SQL query provides the option to split the queried data into chunks (or pages), and it returns only the first page. 

The next cell runs a simple join SQL. But this time `submit_sql()` is provided the optional **`pagesize`** parameter with a value of **`10`**. This results in multiple objects being written with each having 10 rows of the result in it. This optional **`pagesize`** parameter is also available in other related APIs: `run`, `run_sql`, `submit`, `submit_and_track_sql`, `execute_sql`.

Internally this is achieved by using the SQL Query syntax clause of `PARTITIONED EVERY <num> ROWS`. This also means that your query cannot already contain another `PARTITIONED BY` clause.

In [29]:
pagination_sql='SELECT OrderID, c.CustomerID CustomerID, CompanyName, City, Region, PostalCode \
                FROM cos://us-geo/sql/orders.parquet STORED AS PARQUET o, \
                     cos://us-geo/sql/customers.parquet STORED AS PARQUET c \
                WHERE c.CustomerID = o.CustomerID \
                INTO {}paginated_orders STORED AS PARQUET'.format(targeturl)
formatted_etl_sql = sqlparse.format(pagination_sql, reindent=True, indent_tabs=True, keyword_case='upper')
result = highlight(formatted_etl_sql, lexer, formatter)
print('\nExample ETL Statement is:\n')
print(result)

jobId = sqlClient.submit_sql(pagination_sql, pagesize=10)
job_status = sqlClient.wait_for_job(jobId)
print("Job " + jobId + " terminated with status: " + job_status)
job_details = sqlClient.get_job(jobId)
if job_status == 'failed':
    print("Error: {}\nError Message: {}".format(job_details['error'], job_details['error_message']))


Example ETL Statement is:

SELECT OrderID,
	c.CustomerID CustomerID,
	CompanyName,
	City,
	Region,
	PostalCode
FROM cos://us-geo/SQL/orders.parquet STORED AS PARQUET o,
	cos://us-geo/SQL/customers.parquet STORED AS PARQUET c
WHERE c.CustomerID = o.CustomerID INTO cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/paginated_orders STORED AS PARQUET

Job 8a47b930-2df4-45e1-88b7-45b08c48dd72 terminated with status: completed


Let's check how many pages with each 10 rows have been written:


In [30]:
print("Number of pages written by job {}: {}".format(jobId, len(sqlClient.list_results(jobId))))

Number of pages written by job 8a47b930-2df4-45e1-88b7-45b08c48dd72: 85


The following cell retrieves the first page of the result as a data frame. The desired page is specified as the optional parameter **`pagenumber`** to the `get_result()` method.


In [31]:
pagenumber=1
sqlClient.get_result(jobId, pagenumber=pagenumber).head(100)

,OrderID,CustomerID,CompanyName,City,Region,PostalCode
0,11011,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
1,10952,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
2,10835,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
3,10702,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
4,10692,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
5,10643,ALFKI,Alfreds Futterkiste,Berlin,NULL,12209
6,10926,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
7,10759,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
8,10625,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021
9,10308,ANATR,Ana Trujillo Emparedados y helados,México D.F.,NULL,05021


The following cell gets the next page. Run it multiple times in order to retrieve the subsequent pages, one page after the another.


In [51]:
pagenumber+=1
sqlClient.get_result(jobId, pagenumber).head(100)


,OrderID,CustomerID,CompanyName,City,Region,PostalCode
0,10856,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
1,10682,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
2,10677,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
3,10573,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
4,10535,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
5,10507,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
6,10365,ANTON,Antonio Moreno Taquería,México D.F.,NULL,05023
7,11016,AROUT,Around the Horn,London,NULL,WA1 1DP
8,10953,AROUT,Around the Horn,London,NULL,WA1 1DP
9,10920,AROUT,Around the Horn,London,NULL,WA1 1DP


### <a id="many_queries"></a> 10. Automate SQL statements


Suppose that you have many SQL statements that you want to launch, you can program the job launches via a loop.

Because there is a limit of number of queries that can be served by a Data Engine instance at a time, there is an option to control the number of retries using `maxtries` option when creating the `sqlClient` object. 

Also, since `ibmcloudsql` 0.4, if you're using Watson Studio, you have the option to connect to the Project-Lib, and save the list of jobs launched to the given filename, so that when you rerun the session, the progress can be continued. 



In [62]:
file_name = "my_jobs"
sqlClient.connect_project_lib(project, file_name)


NameError: name 'project' is not defined

### <a id="joblist"></a> 11. Working with your SQL Job Submission History
The following cell uses the `get_cos_summary()` method to get a statistical overview of the data in the **target location** in COS that has been used by the above queries in this notebook.

In [61]:
sqlClient.get_cos_summary(targeturl)

{'url': 'cos://us-geo/timeseriessql-donotdelete-pr-flr5gmyhe2adeo/',
 'total_objects': 1734,
 'total_volume': '732.5 MB',
 'oldest_object_timestamp': 'October 16, 2018, 21H:26M:21S',
 'newest_object_timestamp': 'July 26, 2020, 21H:34M:02S',
 'smallest_object_size': '0.0 B',
 'smallest_object': 'customer_orders/jobid=11a8ff2d-6045-4cc1-bdf0-6374aaa85d58',
 'largest_object_size': '234.1 MB',
 'largest_object': 'Omniture.tsv'}

### <a id="next"></a> 12. Next steps
In this notebook you learned how you can use the `ibmcloudsql` library in a Python notebook to submit SQL queries on data in IBM Cloud Object Storage and how you can interact with the query results. If you want to automate such an SQL query execution as part of your cloud solution, apart from the solution using ProjectLib, you can use the <a href="https://console.bluemix.net/openwhisk/" target="_blank">IBM Cloud Functions</a> framework. There is a dedicated SQL function available that lets you set up a cloud function to run SQL statements with IBM Cloud SQL Query. You can find the documentation for doing this <a href="https://hub.docker.com/r/ibmfunctions/sqlquery/" target="_blank" rel="noopener noreferrer">here</a>.

### <a id="authors"></a>Authors

**Torsten Steinbach**, Torsten is the lead architect for IBM Cloud Data Lake. Previously he has worked as IBM architect for a series of data management products and services, including DB2, PureData for Analytics and Db2 on Cloud.

**Tuan M. HoangTrong**, Tuan is the research staff member in the Distributed AI, TimeSeries Group. 